In [2]:
from collections import namedtuple, deque
import random
import gym
import numpy as np

import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.optim import Adam

In [3]:
dq = deque(maxlen=2)

In [4]:
dq.append(3)

In [5]:
dq[-1]

3

In [6]:
??deque

In [7]:
dq.pop()

3

In [8]:
dq

deque([])

In [9]:
?deque

In [10]:
list(dq)

[]

In [11]:
env = gym.make('CartPole-v0')

In [12]:
action_n = env.action_space.n
observation_shape = env.observation_space.shape

In [13]:
env.reset()
for _ in range(100):
    obs, reward, done, _ = env.step(env.action_space.sample())
    if done:
        print(reward)
        break

1.0


In [14]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

# class ReplayMemory:

#     def __init__(self, capacity):
#         self.capacity = capacity
#         self.memory = []
#         self.position = 0

#     def push(self, *args):
#         """Saves a transition."""
#         if len(self.memory) < self.capacity:
#             self.memory.append(None)
#         self.memory[self.position] = Transition(*args)
#         self.position = (self.position + 1) % self.capacity

#     def sample(self, batch_size):
#         return random.sample(self.memory, batch_size)

#     def __len__(self):
#         return len(self.memory)
    
    
def clip_grads(net, low=-10, high=10):
    """Gradient clipping to the range [low, high]."""
    parameters = [param for param in net.parameters()
                  if param.grad is not None]
    for p in parameters:
        p.grad.data.clamp_(low, high)

In [15]:
memory = ReplayMemory(10000)

In [18]:
# count()

In [24]:
np.random.sample([3,4,5,5])

array([[[[  3.33739912e-01,   2.26815365e-02,   5.23401094e-02,
            1.28788046e-01,   2.93179580e-01],
         [  5.69269105e-01,   9.18954875e-01,   2.31940962e-02,
            6.63372018e-01,   9.54840583e-02],
         [  2.56929279e-01,   3.24587362e-01,   9.38760711e-01,
            6.45470278e-01,   5.33332938e-02],
         [  4.15465743e-01,   3.92613006e-01,   3.81625965e-01,
            9.23001802e-01,   7.81511428e-02],
         [  2.89778031e-01,   5.17493987e-01,   7.66651330e-01,
            5.07008136e-01,   5.33344162e-01]],

        [[  2.92642043e-01,   9.94649881e-01,   9.40645628e-01,
            1.04749020e-01,   4.36277621e-01],
         [  6.14243065e-01,   1.25136161e-01,   6.31632624e-01,
            6.60572063e-01,   8.42679829e-01],
         [  3.60856321e-01,   3.67307613e-01,   7.11276935e-01,
            5.92782347e-01,   4.95088867e-01],
         [  6.86984254e-01,   7.81127921e-01,   9.18237022e-01,
            2.73962003e-01,   1.99881055e-01],

In [31]:
random.random()

0.13404598995251682

In [32]:
if torch.cuda.is_available():
    def to_var(x, requires_grad=False, gpu=None):
        x = x.cuda(gpu)
        return Variable(x, requires_grad=requires_grad)
else:
    def to_var(x, requires_grad=False, gpu=None):
        return Variable(x, requires_grad=requires_grad)

In [603]:
class VanillaDQN:
    def __init__(self, model, gamma=0.99, learning_rate=1.e-3, memory_size=5000, action_n=action_n):
        self.model = model
        self.memory = deque(maxlen=memory_size)
        self.gamma = gamma
        self.action_n = action_n
#         self.loss_fn = nn.SmoothL1Loss()
        self.loss_fn = nn.MSELoss()

        self.optimizer = Adam(model.parameters(), lr=learning_rate)
        
    def select_action(self,obs):
        thresh = 0.1
        if np.random.random()<thresh:
            print("Exploration!")
            action = np.random.randint(self.action_n)
        else:
            state = to_var(torch.from_numpy(obs).float().unsqueeze(0))
            q_values = self.get_q_value(state)
            _, action_ = q_values.max(1)
            action = action_.data[0]
        return action
    
    def get_q_value(self, state):
        with torch.no_grad():   
            values = self.model(state)
        return values
    
    def memorize(self, state, action, next_state, reward):
        self.memory.append((state, action, next_state, reward))
        
    def replay(self, batch_size):
        batch = random.sample(self.memory, batch_size)
        
        state_batch = to_var(torch.stack([torch.Tensor(b[0]) for b in batch]))
        
        action_batch = to_var(torch.stack([torch.LongTensor([b[1]]) for b in batch]))
        
        next_state_batch = to_var(torch.stack([torch.Tensor(b[2]) for b in batch if b[2] is not None]))
        non_final_mask = torch.ByteTensor([b[2] is not None for b in batch])
        
        reward_batch = to_var(torch.stack([torch.Tensor([b[3]]) for b in batch]))
        
        curr_values = self.model(state_batch).gather(1,action_batch)
        
        next_values = to_var(torch.zeros(batch_size,1).float())
        next_values[non_final_mask,0] = self.get_q_value(next_state_batch).max(1)[0]
        
        expected_values = next_values*self.gamma+reward_batch
        
        return self.loss_fn(curr_values,expected_values)
        
    def train(self, batch_size):
        if len(self.memory)>batch_size:
            loss = self.replay(batch_size)
            self.optimizer.zero_grad()
            loss.backward()
            clip_grads(self.model,-5,5)
            self.optimizer.step()
            return loss.data[0]
        else:
            print("Not enough experience.")

In [618]:
net = nn.Sequential(nn.Linear(observation_shape[0],128),nn.ReLU(),nn.Linear(128,action_n))

In [619]:
agent = VanillaDQN(model=net)

In [620]:
# ?agent.loss_fn

In [621]:
agent.optimizer.zero_grad()

In [622]:
obs = env.reset()

In [623]:
# trainer.select_action(obs)

In [624]:
# trainer.get_q_value(obs)

In [625]:
net

Sequential(
  (0): Linear(in_features=4, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=2, bias=True)
)

In [626]:
for episode in range(10):
    obs = env.reset()
    total_reward = 0
    for _ in range(10000): # not exceed 10000 episodes
        action = agent.select_action(obs)
        next_obs, reward, done, _ = env.step(action)
        total_reward+=reward
        if done:
            next_obs = None
            agent.memorize(obs, action, None, reward)
            break
        else:
            agent.memorize(obs, action, next_obs, reward)
            obs = next_obs
#     policy_loss = trainer.get_loss_and_clear()
# #     print(policy_loss)
#     optimizer.zero_grad()
#     policy_loss.backward()
#     clip_grads(trainer.model,-10,10)
# #     nn.utils.clip_grad_norm(trainer.model.parameters(), 40)
#     optimizer.step()
#     running_reward = trainer.running_reward
    print(episode, total_reward)
#     if running_reward>env.spec.reward_threshold:
#         break
# print("Finished: %s@%s" %(trainer.running_reward,episode))

Exploration!
0 11.0
1 10.0
Exploration!
Exploration!
Exploration!
2 10.0
Exploration!
3 10.0
Exploration!
4 9.0
Exploration!
5 8.0
Exploration!
Exploration!
6 13.0
7 9.0
Exploration!
8 9.0
Exploration!
9 10.0


In [627]:
agent.memory

deque([(array([-0.00278879, -0.04244314, -0.02409147,  0.03782665]),
        0,
        array([-0.00363766, -0.23721148, -0.02333494,  0.32281217]),
        1.0),
       (array([-0.00363766, -0.23721148, -0.02333494,  0.32281217]),
        0,
        array([-0.00838189, -0.43199351, -0.0168787 ,  0.60804582]),
        1.0),
       (array([-0.00838189, -0.43199351, -0.0168787 ,  0.60804582]),
        0,
        array([-0.01702176, -0.62687546, -0.00471778,  0.89536502]),
        1.0),
       (array([-0.01702176, -0.62687546, -0.00471778,  0.89536502]),
        1,
        array([-0.02955927, -0.43168986,  0.01318952,  0.60120285]),
        1.0),
       (array([-0.02955927, -0.43168986,  0.01318952,  0.60120285]),
        0,
        array([-0.03819306, -0.62699381,  0.02521358,  0.8980109 ]),
        1.0),
       (array([-0.03819306, -0.62699381,  0.02521358,  0.8980109 ]),
        0,
        array([-0.05073294, -0.82244828,  0.04317379,  1.19851143]),
        1.0),
       (array([-0.0507

In [628]:
agent.replay(8)

Variable containing:
 0.9405
[torch.FloatTensor of size 1]

In [629]:
for _ in range(100):
    print(agent.train(8))

0.9628039002418518
1.0617287158966064
0.9399638772010803
0.9645430445671082
1.0721133947372437
1.0933808088302612
0.8353238701820374
1.1272661685943604
1.1220122575759888
1.3189899921417236
1.0171788930892944
1.003360390663147
1.1330327987670898
1.1497286558151245
1.1975243091583252
1.2326909303665161
1.0267066955566406
1.3733447790145874
1.2253601551055908
1.1601336002349854
1.1740061044692993
1.1186258792877197
1.1753649711608887
1.152755856513977
1.4759796857833862
1.1437959671020508
1.240262508392334
1.5769577026367188
1.2217497825622559
1.3926961421966553
1.2804104089736938
1.1859514713287354
1.18784499168396
1.3329658508300781
1.3687474727630615
1.3961527347564697
1.7400516271591187
1.2415223121643066
2.25234317779541
1.7601902484893799
1.2975636720657349
1.4573999643325806
2.252758741378784
1.9759782552719116
1.9269553422927856
1.8858277797698975
2.413299322128296
1.3815866708755493
1.9733426570892334
1.5022735595703125
3.303309440612793
2.005099296569824
3.673975706100464
1.406

In [601]:
agent.select_action(obs)

0

In [602]:
agent.model(curr_state)

Variable containing:
  567.4605   348.0985
  302.0856   185.5246
  693.1074   425.0811
  427.8733   262.5829
  307.6658   188.9338
  820.9556   503.3987
  836.8867   513.1587
 1266.4009   776.4229
[torch.FloatTensor of size 8x2]

In [549]:
net[2].weight

Parameter containing:

Columns 0 to 9 
 0.2772 -0.0306  0.4404  0.4062  0.0396  0.3918  0.3689  0.4046  0.4232  0.0466
 0.1518  0.0054  0.3927  0.2785  0.0236  0.2159  0.2426  0.2888  0.3224  0.0045

Columns 10 to 19 
 0.3930  0.3860  0.2087  0.3062  0.3540  0.3974  0.3929  0.3619  0.4629  0.4138
 0.1483  0.1655  0.1886  0.1467  0.1882  0.1299  0.2058  0.3205  0.1777  0.1574

Columns 20 to 29 
 0.0405  0.3653 -0.0393  0.4434  0.3523  0.3389  0.5083  0.3499  0.4098  0.4431
 0.0460  0.2191 -0.0090  0.3888  0.2443  0.2778  0.2688  0.2011  0.2657  0.2587

Columns 30 to 39 
 0.4230  0.2293  0.2123  0.0490 -0.0618  0.3695  0.2790  0.3699  0.0248  0.0668
 0.3256  0.2062  0.1920 -0.0456  0.0143  0.2974  0.2076  0.3307 -0.0683  0.0061

Columns 40 to 49 
-0.0850  0.3539 -0.0163  0.0476  0.4608  0.3522  0.4010  0.3188  0.4653 -0.0738
 0.0634  0.2453  0.0239 -0.0821  0.1959  0.1247  0.1649  0.2866  0.2383 -0.0358

Columns 50 to 59 
 0.4361  0.4720  0.3614 -0.0387  0.4020  0.4541  0.2742  0.3706  0

In [424]:
batch_size=  8
batch = random.sample(trainer.memory,batch_size)
batch

[(array([-0.04370104,  0.04001198, -0.01649731,  0.017633  ]),
  0,
  array([-0.09739085, -1.1308732 ,  0.07244025,  1.77986859]),
  1.0),
 (array([-0.07134001, -1.52045395,  0.20839385,  2.53939648]), 0, None, 1.0),
 (array([-0.00286265, -0.04761592,  0.02173505, -0.00883143]),
  0,
  array([-0.00381497, -0.24304273,  0.02155842,  0.29062914]),
  1.0),
 (array([-0.03012432, -0.82959048,  0.05699586,  1.19594499]),
  0,
  array([-0.04671613, -1.02540209,  0.08091476,  1.50593317]),
  1.0),
 (array([ 0.04748833, -0.00517985,  0.03602555, -0.02206806]),
  1,
  array([ 0.04738473,  0.18940743,  0.03558419, -0.30317038]),
  1.0),
 (array([-0.10070778, -1.19688313,  0.13848175,  1.80594401]),
  1,
  array([-0.12464545, -1.00355224,  0.17460063,  1.55930753]),
  1.0),
 (array([ 0.03825429, -0.42009394, -0.02331374,  0.60132773]),
  0,
  array([ 0.02985241, -0.61488213, -0.01128719,  0.88657701]),
  1.0),
 (array([-0.00991224, -0.03945237, -0.01121769,  0.02134875]),
  0,
  array([-0.15756548

In [331]:
curr_state = to_var(torch.stack([torch.Tensor(b[0]) for b in batch]))

In [332]:
[torch.Tensor(b[0]) for b in batch][0]


 0.0287
-0.3444
 0.0402
 0.6227
[torch.FloatTensor of size 4]

In [333]:
curr_state

Variable containing:
 0.0287 -0.3444  0.0402  0.6227
 0.0339 -0.0069 -0.0353 -0.0137
 0.0299 -0.6149 -0.0113  0.8866
-0.0203 -0.2270 -0.0320  0.2365
 0.0095  0.0355  0.0460 -0.0234
 0.0086 -0.7322  0.0533  1.2078
 0.0224 -0.7507  0.0725  1.2415
-0.1108 -1.3579  0.1740  2.1837
[torch.FloatTensor of size 8x4]

In [334]:
curr_values = trainer.model(curr_state)

In [336]:
curr_values.max(1)[0]

Variable containing:
 0.2722
 0.2027
 0.2926
 0.2099
 0.1931
 0.3323
 0.3368
 0.5037
[torch.FloatTensor of size 8]

In [337]:
action_batch = to_var(torch.stack([torch.LongTensor([b[1]]) for b in batch]))

In [338]:
reward_batch = to_var(torch.stack([torch.Tensor([b[3]]) for b in batch]))

In [339]:
# to_var(torch.stack([torch.LongTensor([b[3]]) for b in batch]))

In [340]:
actions = to_var(torch.LongTensor([b[1] for b in batch])).unsqueeze(-1)

In [341]:
actions.data[3] = 1

In [342]:
actions

Variable containing:
    0
    0
    1
    1
    0
    1
    0
    0
[torch.LongTensor of size 8x1]

In [343]:
curr_values

Variable containing:
 0.2722  0.1775
 0.2027  0.1657
 0.2926  0.2155
 0.2099  0.1831
 0.1931  0.1539
 0.3323  0.2575
 0.3368  0.2633
 0.5037  0.4775
[torch.FloatTensor of size 8x2]

In [344]:
curr_values.gather(1, actions)

Variable containing:
 0.2722
 0.2027
 0.2155
 0.1831
 0.1931
 0.2575
 0.3368
 0.5037
[torch.FloatTensor of size 8x1]

In [345]:
next_states = to_var(torch.stack([torch.Tensor(b[2]) for b in batch if b[2] is not None]))
non_final_mask = torch.ByteTensor([b[2] is not None for b in batch])

In [346]:
non_final_mask


 1
 1
 1
 1
 1
 1
 1
 0
[torch.ByteTensor of size 8]

In [347]:
trainer.get_q_value(next_states).max(1)[0]

Variable containing:
 0.3034
 0.2176
 0.2613
 0.2479
 0.5814
 0.3007
 0.3875
[torch.FloatTensor of size 7]

In [350]:
next_values = to_var(torch.zeros(batch_size,1).float())
next_values[non_final_mask,0] = trainer.get_q_value(next_states).max(1)[0]

In [351]:
next_values

Variable containing:
 0.3034
 0.2176
 0.2613
 0.2479
 0.5814
 0.3007
 0.3875
 0.0000
[torch.FloatTensor of size 8x1]

In [266]:
gamma=0.99

In [268]:
expected_values = next_values*0.99+reward_batch
expected_values

Variable containing:
 1.2093
 1.6372
 1.2125
 1.0000
 1.3624
 1.3016
 1.3168
 1.0000
[torch.FloatTensor of size 8x1]

In [269]:
curr_values.gather(1,action_batch)

Variable containing:
 0.1951
 0.5600
 0.1957
 0.4354
 0.1988
 0.2727
 0.2840
 0.5716
[torch.FloatTensor of size 8x1]

In [270]:
curr_values.gather(1,action_batch)-expected_values

Variable containing:
-1.0142
-1.0772
-1.0168
-0.5646
-1.1636
-1.0290
-1.0328
-0.4284
[torch.FloatTensor of size 8x1]

In [271]:
curr_values.gather(1,action_batch)

Variable containing:
 0.1951
 0.5600
 0.1957
 0.4354
 0.1988
 0.2727
 0.2840
 0.5716
[torch.FloatTensor of size 8x1]

In [272]:
expected_values

Variable containing:
 1.2093
 1.6372
 1.2125
 1.0000
 1.3624
 1.3016
 1.3168
 1.0000
[torch.FloatTensor of size 8x1]

In [273]:
trainer.loss_fn(curr_values.gather(1,action_batch),expected_values)

Variable containing:
 0.4481
[torch.FloatTensor of size 1]

In [201]:
# ?torch.gather

In [237]:
curr_values

Variable containing:
 0.1951  0.1609
 0.5600  0.5571
 0.1957  0.1602
 0.4354  0.4138
 0.1988  0.1686
 0.2727  0.2175
 0.2840  0.2156
 0.5716  0.5693
[torch.FloatTensor of size 8x2]

In [239]:
action_batch[:,0]

Variable containing:
 0
 0
 0
 0
 0
 0
 0
 0
[torch.LongTensor of size 8]

In [244]:
curr_values.gather(1,action_batch[:,0])

RuntimeError: invalid argument 4: Index tensor must have same dimensions as input tensor at /home/jiancheng/apt/pytorch/aten/src/TH/generic/THTensorMath.c:513